# 10.2 Handoff 与 Supervisor + Worker

本笔记本演示如何使用 OpenAI Agents SDK 实现 Handoff。

**预估成本**: ~$0.10

In [ ]:
!pip install openai -q

In [ ]:
import os
from openai import OpenAI

client = OpenAI()
print('✓ OpenAI client initialized')

## Demo 1: 创建 Worker Agents

In [ ]:
# Worker 1: Order Agent
order_agent = client.beta.assistants.create(
    name='Order Specialist',
    instructions='Handle order-related queries. Hand off to billing for refunds.',
    model='gpt-4o-mini',
    tools=[],
    handoffs=[{'type': 'handoff', 'target': 'billing_agent'}],
)

# Worker 2: Billing Agent  
billing_agent = client.beta.assistants.create(
    name='Billing Specialist',
    instructions='Handle payment and refund issues.',
    model='gpt-4o-mini',
    tools=[],
    handoffs=[{'type': 'handoff', 'target': 'order_agent'}],
)

print(f'Order Agent: {order_agent.id}')
print(f'Billing Agent: {billing_agent.id}')

## Demo 2: 创建 Supervisor

In [ ]:
receptionist = client.beta.assistants.create(
    name='Receptionist',
    instructions='Route customer requests to the right specialist.',
    model='gpt-4o-mini',
    handoffs=[
        {'type': 'handoff', 'target': 'order_agent'},
        {'type': 'handoff', 'target': 'billing_agent'},
    ],
)

print(f'Receptionist: {receptionist.id}')

## Demo 3: 模拟 Handoff 流程

In [ ]:
def simulate_handoff(user_message):
    print('=== Handoff 流程 ===')
    print(f'User: {user_message}\n')
    
    print('[Step 1] Receptionist 分析请求')
    print('  判断: 这是退款问题')
    print('  决定: Handoff to billing_agent\n')
    
    print('[Step 2] Billing Agent 接管')
    print('  处理退款请求')
    print('  可能需要订单信息，Handoff to order_agent\n')
    
    print('[Step 3] Order Agent 提供订单详情')
    print('  返回订单信息')
    print('  Handoff back to billing_agent\n')
    
    print('[Step 4] Billing Agent 完成退款')
    print('  最终回复用户\n')
    
    return 'Refund processed successfully'

result = simulate_handoff('我想退款订单 #12345')
print(f'Result: {result}')

## Demo 4: Handoff 决策逻辑

In [ ]:
def demonstrate_handoff_decision():
    print('=== Handoff 决策示例 ===')
    
    scenarios = [
        {
            'message': '订单发货了吗？',
            'target': 'order_agent',
            'reason': '订单状态查询',
        },
        {
            'message': '我要退款',
            'target': 'billing_agent',
            'reason': '退款请求',
        },
        {
            'message': 'API 报错了',
            'target': 'tech_agent',
            'reason': '技术问题',
        },
    ]
    
    for scenario in scenarios:
        print(f"\n消息: {scenario['message']}")
        print(f"→ Handoff to: {scenario['target']}")
        print(f"  原因: {scenario['reason']}")

demonstrate_handoff_decision()

## 总结

- Handoff 是多 Agent 协作的核心机制
- Supervisor + Worker 是经典模式
- Agent 自主决定何时 Handoff
- 上下文自动传递

下一节构建完整的研究系统！